In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
# config

overlap = 20
patch_size = [96, 96, 96]
# initialize 3d image 256*256*256, filled with random values
image = np.random.rand(256, 256, 256)

In [ ]:
# precprocessing
# convert to 8 bit

# get patch 96 * 96 * 96 from image with overlap 20
def get_patch(image, patch_size, overlap):
    patches = []
    for i in range(0, image.shape[0], patch_size[0] - overlap):
        for j in range(0, image.shape[1], patch_size[1] - overlap):
            for k in range(0, image.shape[2], patch_size[2] - overlap):
                patch = image[i:i + patch_size[0], j:j + patch_size[1], k:k + patch_size[2]]
                patches.append(patch)
    return patches

# filter function
# calcualte average intensity, determine whether to keep it base on the random number we generated
# make the threshold 20 (90% black), everything below generate random number 

# tri linear downsampling
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.zoom.html
# zoom image to 1/3 and 3 of the size


In [ ]:
# augmentation libraries: monai, kornia, volumnetations; https://github.com/kornia/kornia
# when dataloader 
# implement them ourselves: translation, rotation (90 degrees), flipping/mirroring, small brighteness/contrast

In [ ]:
# dataset
class CustomImageDataset(Dataset):
    def __init__(high_res_img, low_res_img):
        pass

    def __len__(self):
        pass

    def __getitem__(self, idx):
        pass